# kutubxonalarni chaqiramiz

In [ ]:
import pandas as pd
import numpy as np

from joblib import dump
import os

from sklearn.model_selection import train_test_split, KFold,cross_val_score
from sklearn.feature_selection import RFE

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

Xgb_Clas = XGBClassifier(use_label_encoder=False, eval_metrics='mlogloss')
N_neig = KNeighborsClassifier()
DT_Clas = DecisionTreeClassifier()
RF_Clas = RandomForestClassifier()
SV_Clas = SVC()
LR_Clas = LogisticRegression()

from sklearn.metrics import accuracy_score

# Datasetni yuklaymiz

In [5]:
df = pd.read_csv(r"C:\Users\bunyo\OneDrive\Desktop\git_project\2_week_project\data\preprocessed_data\preprocessed_data.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22899 entries, 0 to 22898
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   latitude                        22899 non-null  float64
 1   longitude                       22899 non-null  float64
 2   mag                             22899 non-null  float64
 3   clustering_coefficient_30_days  22899 non-null  float64
 4   std_mag_30_days                 22899 non-null  float64
 5   rolling_mean_depth_30_days      22899 non-null  float64
 6   earthquakes_last_30_days        22899 non-null  float64
 7   b_value                         22899 non-null  float64
 8   b_value_increment_i_i2          22899 non-null  float64
 9   b_value_increment_i2_i4         22899 non-null  float64
 10  b_value_increment_i4_i6         22899 non-null  float64
 11  b_value_increment_i6_i8         22899 non-null  float64
 12  b_value_increment_i8_i10        

# Filter bilan eng yaxshi feturelarni olamiz

In [10]:
corr_Df = df.corr()['class']
corr_Df

latitude                          0.028836
longitude                        -0.073590
mag                               0.036746
clustering_coefficient_30_days   -0.005725
std_mag_30_days                   0.024852
rolling_mean_depth_30_days       -0.058665
earthquakes_last_30_days         -0.009482
b_value                          -0.035609
b_value_increment_i_i2            0.000415
b_value_increment_i2_i4           0.000202
b_value_increment_i4_i6           0.000345
b_value_increment_i6_i8           0.000534
b_value_increment_i8_i10          0.000439
max_mag_last_week                 0.036399
eta                               0.037867
delta_M                          -0.007755
elapsed_time                     -0.011789
coefficient_of_variation          0.075612
dE1_2                             0.124231
class                             1.000000
mag_depth_ratio                   0.004803
recent_activity_rate              0.010731
Name: class, dtype: float64

In [11]:
corr_df = corr_Df[abs(corr_Df) > 0.01]
corr_df

latitude                      0.028836
longitude                    -0.073590
mag                           0.036746
std_mag_30_days               0.024852
rolling_mean_depth_30_days   -0.058665
b_value                      -0.035609
max_mag_last_week             0.036399
eta                           0.037867
elapsed_time                 -0.011789
coefficient_of_variation      0.075612
dE1_2                         0.124231
class                         1.000000
recent_activity_rate          0.010731
Name: class, dtype: float64

# kerakli ustunlarni saqlab qoyamiz

In [12]:
df_corr = df[corr_df.index]

# -----------------------------------------------------------------

 # ikki xil yol tutamiz filter methodi bilan va filtersiz model yasaymiz, bunga sabab
 # filter qisak kerakli ustunlarni ham tashab ketyapti, shuning uchun wraper va embeded dan foydalanamiz

# -----------------------------------------------------------------



# datasetni bolamiz(filtersiz)

In [13]:
x = df.drop('class',axis=1)
y = df['class']

# filter ishlatilgan datasetni bolamiz

In [14]:
x1 = df_corr.drop('class',axis=1)
y1 = df_corr['class']

# train test ga ajratamiz(filtersiz datani)

In [15]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.23, random_state=99)

# filter ishlatilgan datani ajratamiz

In [16]:
x1_train,x1_test,y1_train,y1_test = train_test_split(x1,y1, test_size=0.2, random_state=100)

k_fold = KFold(n_splits=6,shuffle=True, random_state=99)

# ||---  algorithm lar bilan ishlashni boshlaymiz  ---||

#  -----------------------------------------------------
# dastlab filter methodi ishlatilmagan data bilan koramiz
#  -----------------------------------------------------



# DECISION TREE

In [38]:
Rfe = RFE(DT_Clas, n_features_to_select=10)
Rfe.fit(x_train, y_train)
selected_features = x_train.columns[Rfe.support_]

DT_Clas.fit(x_train[selected_features], y_train)
pred4 = DT_Clas.predict(x_test[selected_features])

crosValScore = cross_val_score(Rfe, x, y, cv=k_fold)

print(f" accuracy: {accuracy_score(y_test, pred4)}")
print(f"KFold: {crosValScore.mean()}")

 accuracy: 0.9145623694702867
KFold: 0.9139261591683973


#  RANDOM FOREST

In [39]:
Rfe = RFE(RF_Clas, n_features_to_select=10) 
Rfe.fit(x_train, y_train)
selected_features = x_train.columns[Rfe.support_]

RF_Clas.fit(x_train[selected_features], y_train)
pred3 = RF_Clas.predict(x_test[selected_features])

crosValScore = cross_val_score(Rfe, x, y, cv=k_fold)

print(f" accuracy: {accuracy_score(y_test, pred3)}")
print(f"KFold: {crosValScore.mean()}")

 accuracy: 0.9821530282893488
KFold: 0.979999263565274


# XGBoost

In [17]:
Rfe = RFE(Xgb_Clas, n_features_to_select=8)
Rfe.fit(x_train, y_train)
selected_features = x_train.columns[Rfe.support_]

Xgb_Clas.fit(x_train[selected_features], y_train)
pred2 = Xgb_Clas.predict(x_test[selected_features])

crosValScore = cross_val_score(Rfe, x, y, cv=k_fold)

print(f" accuracy: {accuracy_score(y_test, pred2)}")
print(f"KFold: {crosValScore.mean()}")

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [1 2 3 4 5 6]

# Gradient Boosting

In [41]:
grBoost = GradientBoostingClassifier(n_estimators=9, max_depth=3, random_state=99)

Rfe = RFE(grBoost, n_features_to_select=10)
Rfe.fit(x_train, y_train)
SelectedFeatures = x_train.columns[Rfe.support_]

grBoost.fit(x_train[SelectedFeatures], y_train)
pred1 = grBoost.predict(x_test[SelectedFeatures])

crosValScore = cross_val_score(Rfe, x, y, cv=k_fold)

print(f" accuracy: {accuracy_score(y_test, pred1)}")
print(f"KFold: {crosValScore.mean()}")

 accuracy: 0.43933928232390357
KFold: 0.4528145354364701


# KNN

In [29]:
N_neig.fit(x_train,y_train)

KNeighborsClassifier()

In [31]:
pred5 = N_neig.predict(x_test)
crosValScore = cross_val_score(N_neig, x, y, cv=k_fold)


print(f" accuracy: {accuracy_score(y_test, pred5)}")
print(f"KFold: {crosValScore.mean()}")

 accuracy: 0.6360356939434213
KFold: 0.6378438289699232


# Logistic regression

In [41]:
lin_model = LogisticRegression(multi_class='multinomial',solver='saga')
lin_model.fit(x_train,y_train)

c:\Users\bunyo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\bunyo\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(multi_class='multinomial', solver='saga')

In [42]:
pred = lin_model.predict(x_test)

crosValScore = cross_val_score(LR_Clas, x, y, cv=k_fold)


print(f" accuracy: {accuracy_score(y_test, pred)}")
print(f"KFold: {crosValScore.mean()}")

c:\Users\bunyo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\bunyo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

 accuracy: 0.25802164419973417
KFold: 0.30503481061498566


c:\Users\bunyo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# SVM

In [45]:
Rfe = RFE(SV_Clas, n_features_to_select=10)
Rfe.fit(x_train, y_train)
SelectedFeatures = x_train.columns[Rfe.support_]

SV_Clas.fit(x_train[SelectedFeatures], y_train)
pred1 = SV_Clas.predict(x_test[SelectedFeatures])

crosValScore = cross_val_score(Rfe, x, y, cv=k_fold)

print(f" accuracy: {accuracy_score(y_test, pred1)}")
print(f"KFold: {crosValScore.mean()}")

: 

: 

SVM wrapper methodi bilan ishlashga qiynaldi 20 minutda ham chiqmadi

# tabulate

In [ ]:
from tabulate import tabulate

In [ ]:
results = [
    ['Logistic Regression','0.258','0.305'],
    ['Decision Tree', '0.914','913'],
    ['Random Forest', '0.982','0.979'],
    ['Gradient Boosting', '0.439','0.452'],
    ['SVM', ''],
    ['XGBoost', 'error','error'],
]

headers = ['Algorithm', 'accuracy_score', 'with Kfold']

table = tabulate(results, headers=headers, tablefmt='fancy_grid', floatfmt='.3f')

print(table)

In [ ]:
from joblib import dump

In [ ]:
with open("jadval.csv", "w", encoding="utf-8") as f:
    f.write(table)